In [3]:
import sqlite3
import pandas as pd
import numpy as np

In [4]:
con = sqlite3.connect(':memory:')

In [5]:
data = pd.read_csv('D:\\DS\\DS school\\sql\\boston.csv')
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [6]:
data.to_sql('boston', con, index=False, if_exists='replace')

506

In [7]:
pd.read_sql(
    '''
    SELECT * FROM boston
    limit 5
    ''',
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [8]:
tasks = 0

1. Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)

In [9]:
tasks+=1
pd.read_sql(
    '''
    select (SELECT count() as CRIM FROM boston where CRIM isnull) as CRIM, 
        (SELECT count() as ZN FROM boston where ZN isnull) as ZN,
        (SELECT count() as INDUS FROM boston where INDUS isnull) as INDUS,
        (SELECT count() as CHAS FROM boston where CHAS isnull) as CHAS,
        (SELECT count() as NOX FROM boston where NOX isnull) as NOX
    ''',
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX
0,0,0,0,0,0


Вывод: в выбранных колонках пустых значений не обнаружено

2. Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)

In [10]:
tasks+=1
pd.read_sql(
    '''
    select count(distinct CRIM) as CRIM, 
    count(distinct ZN) as ZN, 
    count(distinct INDUS) as INDUS, 
    count(distinct CHAS) as CHAS, 
    count(distinct NOX) as NOX from boston
    ''',
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX
0,504,26,76,2,81


3. Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX.

Не знаю, является ли это читингом, но проверку на четность/нечетность решил не писать, т.к. мы видим и так, что количество записей четное

In [11]:
tasks+=1
pd.read_sql(
    '''
    select Name from 
    (
    SELECT 'CM' as 'Name', 
    (
    (SELECT AVG(CRIM) from
    (SELECT CRIM FROM boston ORDER BY CRIM LIMIT 2 OFFSET
    (SELECT (COUNT(*) - 2) / 2 FROM boston))) - (SELECT min(CRIM) from boston)
    ) as def
    union
    SELECT 'ZN',
    (
    (SELECT AVG(ZN) from
    (SELECT ZN FROM boston ORDER BY ZN LIMIT 2 OFFSET
    (SELECT (COUNT(*) - 2) / 2 FROM boston))) - (SELECT min(ZN) from boston)
    ) as def
    union
    SELECT 'INDUS',
    (
    (SELECT AVG(INDUS) from
    (SELECT INDUS FROM boston ORDER BY INDUS LIMIT 2 OFFSET
    (SELECT (COUNT(*) - 2) / 2 FROM boston))) - (SELECT min(INDUS) from boston)
    ) as def
    union
    SELECT 'CHAS',
    (
    (SELECT AVG(CHAS) from
    (SELECT CHAS FROM boston ORDER BY CHAS LIMIT 2 OFFSET
    (SELECT (COUNT(*) - 2) / 2 FROM boston))) - (SELECT min(CHAS) from boston)
    ) as def
    union
    SELECT 'NOX',
    (
    (SELECT AVG(NOX) from
    (SELECT NOX FROM boston ORDER BY NOX LIMIT 2 OFFSET
    (SELECT (COUNT(*) - 2) / 2 FROM boston))) - (SELECT min(NOX) from boston)
    ) as def
    ) where def = 0
    ''',
    con,
)

,Name
0,CHAS
1,ZN


4. Выведите разницу между среднем количеством комнат(RM) в домах с самой дорогой стоимостью(MEDV) и 25 самыми дешевыми домами.
Аналогично по 50, 100, 200, 300 самыми дешевыми домами. (кол-во домов(25,50,100,200,300), среднее кол-во комнат в них, среднее кол-во комнат в самых дорогих, разница). Напишите влияет ли кол-во комнат на стоимость и как сильно.

In [12]:
tasks+=1
pd.read_sql(
    '''
    select house, RM_exp, RM_cheap, (RM_exp-RM_cheap) as diff from
    (
    select 25 as 'house', 
    (select avg(RM) from (select rm, medv from boston where MEDV = (select max(MEDV) from boston))) as RM_exp,
    (select avg(RM) from (select rm, MEDV from boston order by MEDV limit 25)) as RM_cheap
    union
    select 50, 
    (select avg(RM) from (select rm, medv from boston where MEDV = (select max(MEDV) from boston))) as RM_exp,
    (select avg(RM) from (select rm, MEDV from boston order by MEDV limit 50)) as RM_cheap
    union
    select 100, 
    (select avg(RM) from (select rm, medv from boston where MEDV = (select max(MEDV) from boston))) as RM_exp,
    (select avg(RM) from (select rm, MEDV from boston order by MEDV limit 100)) as RM_cheap
    union
    select 200, 
    (select avg(RM) from (select rm, medv from boston where MEDV = (select max(MEDV) from boston))) as RM_exp,
    (select avg(RM) from (select rm, MEDV from boston order by MEDV limit 200)) as RM_cheap
    union
    select 300, 
    (select avg(RM) from (select rm, medv from boston where MEDV = (select max(MEDV) from boston))) as RM_exp,
    (select avg(RM) from (select rm, MEDV from boston order by MEDV limit 300)) as RM_cheap
    )
    ''',
    con,
)

,house,RM_exp,RM_cheap,diff
0,25,7.484,5.747840,1.736160
1,50,7.484,5.753240,1.730760
2,100,7.484,5.887120,1.596880
3,200,7.484,5.911705,1.572295
4,300,7.484,5.972227,1.511773


Вывод: можно отметить, что чем дешевле жилье, тем больше разница в количестве комнат по сравнению с самым дорогим. При этом разница в количестве комнат заметно уменьшается при переходе от ТОП-50 до ТОП-100 самого дешевого жилья.

5. Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг) среди всех значений LSTAT. Напишите какой вывод можно сделать по этим данным.

In [13]:
tasks+=1
pd.read_sql(
    '''
    select * from (
    select LSTAT, MEDV, rank() OVER (ORDER BY MEDV DESC) as rank from boston)
    where MEDV = (select max(medv) from boston) 
    ''',
    con,
)

,LSTAT,MEDV,rank
0,1.73,50.0,1
1,1.92,50.0,1
2,3.32,50.0,1
3,3.70,50.0,1
4,4.45,50.0,1
5,2.97,50.0,1
6,2.88,50.0,1
7,4.63,50.0,1
8,5.12,50.0,1
9,7.44,50.0,1


Вывод: корреляции между процентом населения с более низким статусом и стоимостью жилья не прослеживается

6. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [14]:
tasks+=1
pd.read_sql(
    '''
    SELECT CHAS, avg(MEDV) as 'mean_price' FROM boston where CHAS = 0
    union
    SELECT CHAS, avg(MEDV) FROM boston where CHAS = 1
    ''',
    con,
)

,CHAS,mean_price
0,0.0,22.093843
1,1.0,28.440000


Вывод: стоимость жилья в случае, если оно граничит с рекой, существенно повышается

7. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.

Убрал из вывода столбец CHAS, т.к. не совсем понял, зачем он тут - мы сраниваем влияние этого параметра на другие значения, проверять его влияение на самого себя как будто бы не логично

In [15]:
tasks+=1
pd.read_sql(
    '''
    select * from
    (
    select 'CRIM' as Column, 
    ((SELECT avg(CRIM) FROM boston where CHAS = 1) - (SELECT avg(CRIM) FROM boston where CHAS = 0)) as diff_mean
    union
    select 'ZN', 
    ((SELECT avg(ZN) FROM boston where CHAS = 1) - (SELECT avg(ZN) FROM boston where CHAS = 0)) as diff_mean
    union
    select 'INDUS', 
    ((SELECT avg(INDUS) FROM boston where CHAS = 1) - (SELECT avg(INDUS) FROM boston where CHAS = 0)) as diff_mean
    union
    select 'NOX', 
    ((SELECT avg(NOX) FROM boston where CHAS = 1) - (SELECT avg(NOX) FROM boston where CHAS = 0)) as diff_mean
    ) where diff_mean > 0
    ''',
    con,
)

,Column,diff_mean
0,INDUS,1.699950
1,NOX,0.041609


Вывод: доля промышленной застройки выше в тех районах, где дома граничат с рекой, следовательно, повышается и уровень концентрации оксидов азота

8. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

In [16]:
tasks+=1
pd.read_sql(
    '''
    select PCT, AVG(INDUS) as mean_INDUS, AVG(NOX) as mean_NOX from
    (
    select INDUS, NOX, (ntile(10) over(order by INDUS))*10 as PCT from boston
    ) group by PCT
    ''',
    con,
)

,PCT,mean_INDUS,mean_NOX
0,10,2.016078,0.438378
1,20,3.645294,0.494129
2,30,5.246275,0.448235
3,40,6.471765,0.470839
4,50,8.256078,0.531510
5,60,10.749216,0.515176
6,70,17.245800,0.649660
7,80,18.100000,0.675520
8,90,18.307200,0.676480
9,100,21.938800,0.655620


Вывод: очевидна корреляция уровня концентрации оксида азота с долей промышленной застройки в этих районах.

In [17]:
if tasks==8:
    print('Выполнены все задания 🙂')
else:
    print('Выполнены не все задания! 🙀')

Выполнены все задания 🙂


P.S. достаточно редко использовал оконные функции, не уверен, что мои способы решения оптимальные. Если в какой-то задаче нужно использовать именно оконные функции, то могу попробовать переписать код